In [ ]:
import sys
sys.path.append('..')
import elastic_rods
import numpy as np
from typing import NamedTuple
from bending_validation import suppress_stdout as so
from tri_mesh_viewer import TriMeshViewer

import fd_validation

def combine(shearStress, sigma_zz):
    return np.array(list(shearStress) + [sigma_zz])
class StressMeasureFDWrapper():
    def __init__(self, stressType, squared):
        self.shearStress = np.random.uniform(size=2)
        self.sigma_zz = np.random.uniform()
        self.stressType = stressType
        self.squared = squared
    def numVars(self):    return 3
    def getVars(self):    return combine(self.shearStress, self.sigma_zz)
    def setVars(self, x): self.shearStress, self.sigma_zz = x[0:2], x[2]
    def energy(self):     return elastic_rods.CrossSectionStressAnalysis.stressMeasure(self.stressType, self.shearStress, self.sigma_zz, self.squared)
    def gradient(self):   return combine(*elastic_rods.CrossSectionStressAnalysis.gradStressMeasure(self.stressType, self.shearStress, self.sigma_zz, self.squared))
    def name(self):       return self.stressType.name + (' Squared' if self.squared else '')
    
ST = elastic_rods.CrossSectionStressAnalysis.StressType
from matplotlib import pyplot as plt
numSTs = len(ST.__members__.items())
counter = 1
plt.figure(figsize=(14,12))
for name, st in ST.__members__.items():
    plt.subplot(numSTs, 2, counter)
    fd_validation.gradConvergencePlot(StressMeasureFDWrapper(st, True))
    plt.subplot(numSTs, 2, counter + 1)
    fd_validation.gradConvergencePlot(StressMeasureFDWrapper(st, False))
    counter += 2
plt.tight_layout()

In [ ]:
rodWidth = 520
npts = 199
midpt = (npts + 1) // 2
thetaOffset = 3 * npts

In [ ]:
twistAngle = np.pi / 4
x_strain = 0.01
test = 'bend'
#test = 'twist'
#test = 'stretch'
#test = 'stretchtwist'

In [ ]:
# Note: the ellipse cross-section used to construct the tetrahedral mesh below has only 20 subdivisons
# (determined by the "visualization resolution" of `CrossSections::Ellipse`).
# This introduces significant discretization error, making the energy *lower* and
# the stress *higher* than the true values!

In [ ]:
pts = np.pad(np.linspace(-rodWidth / 2, rodWidth / 2, npts)[:,np.newaxis], [(0, 0), (0, 2)], mode='constant')
r = elastic_rods.ElasticRod(pts)
mat = elastic_rods.RodMaterial('rectangle', 2000, 0.3, [12, 8], stiffAxis=elastic_rods.StiffAxis.D2, keepCrossSectionMesh=True)
r.setMaterial(mat)

In [ ]:
rigidMotionVars  = [3 * midpt, 3 * midpt + 2] # pin x and z translation
rigidMotionVars += [2]                        # pin rotation around y axis (z comp. of arbitrary vtx)
rigidMotionVars += [thetaOffset]              # pin rotation around x axis

In [ ]:
x = r.getDoFs()
dirichletVars = []
if (test=='bend'):
    dirichletVars = [1, 3 * midpt + 1, 3 * (npts - 1) + 1, len(x) - 1]
    x[dirichletVars] = [10, -10, 10, 0.0]
if ('twist' in test):
    dirichletVars = [1, len(x) - 1]
    x[dirichletVars] = [0, twistAngle]
if ('stretch' in test):
    rigidMotionVars = rigidMotionVars[1:]
    dirichletVars += [0, 3 * (npts - 1)]
    x[0]              = (1 + x_strain) * -260
    x[3 * (npts - 1)] = (1 + x_strain) *  260
r.setDoFs(x)
fixedVars = rigidMotionVars + dirichletVars

In [ ]:
import py_newton_optimizer
opts = py_newton_optimizer.NewtonOptimizerOptions()
opts.niter = 1000
opts.useIdentityMetric = False
opts.useNegativeCurvatureDirection = True
opts.gradTol = 1e-4
opts.verbose = 0
forces = []
elastic_rods.compute_equilibrium(r, fixedVars=fixedVars, options=opts);

In [ ]:
st = ST.VonMises

In [ ]:
r.set_design_parameter_config(True, True)

In [ ]:
x = r.getExtendedDoFs()

In [ ]:
r.setExtendedDoFs(x + 1e-2 * np.random.normal(size=r.numExtendedDoF()))

In [ ]:
r.updateSourceFrame()

In [ ]:
direction = np.random.normal(size=r.numExtendedDoF())
#direction[r.numDoF():] = 0
#direction[0:r.numDoF()] = 0

In [ ]:
class StressLpNormFDWrapper():
    def __init__(self, stressType, p, r):
        self.stressType = stressType
        self.p = p
        self.r = r
    def numVars(self):    return self.r.numExtendedDoF()
    def getVars(self):    return self.r.getExtendedDoFs()
    def setVars(self, x): self.r.setExtendedDoFs(x)
    def energy(self):     return self.r.surfaceStressLpNorm(self.stressType, self.p, False)
    def gradient(self):   return self.r.gradSurfaceStressLpNorm(self.stressType, self.p, True, False)
    def name(self):       return self.stressType.name + f' L_{self.p} norm'
    
numSTs = len(ST.__members__.items())
counter = 1
plt.figure(figsize=(14,12))
for name, st in ST.__members__.items():
    plt.subplot(numSTs, 2, counter)
    fd_validation.gradConvergencePlot(StressLpNormFDWrapper(st, 2, r), direction)
    plt.subplot(numSTs, 2, counter + 1)
    fd_validation.gradConvergencePlot(StressLpNormFDWrapper(st, 6, r), direction)
    counter += 2
plt.tight_layout()